 

# <p style="text-align: center;">Sentimental Analysis  </p>

                                                             

### Importing required Libraries

In [55]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re

### Reading Data

In [56]:
#importing data file and checking the links
import pandas as pd
link=pd.read_excel(r"C:\Users\heman\OneDrive\Black coffer\Input.xlsx")
link['URL_ID']=link.URL_ID.astype(int)
link.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [57]:
#Reading each link and webscraping the relevant data and saving it in text file with URL_ID as file name

ID=link.URL_ID[0]
for blog in link.URL[:]:
        uClient=uReq(blog)
        page_html=uClient.read()
        uClient.close()
        page_soup=soup(page_html,"html.parser")                        #making html readable using beautiful soup
        title=page_soup.find("h1",attrs={"class":"entry-title"}).text  #extracting title
        containers=page_soup.findAll("p")                              #Finding all containers with p in html
        paragraphs=[]
        for paragraph in containers:
               paragraphs.append(paragraph.get_text(' ',strip=True))   #Reading each paragraph in the link and appending it
        text=[x.lower() for x in paragraphs]        
        text=" ".join(text)                                            #Changing from list to string
        text=re.sub('[;:,\?\"\'\/]','',text)   #Removing all punctuations except ! as this is a sentiment analysis.
        bodytext=title.lower() +" "+ text       #Changing all to lower case
        
        text_link = open(f'{str(ID)}.txt',mode="w",encoding='utf-8')   #Creating a text file with URLID as file name
        text_link.write(bodytext)               #wrting the text to text file
        text_link.close()
        ID+=1              

### Stop Words



In [58]:
#importing glob and reading all the stop words text files

import glob
stop_words_path=glob.glob(r"C:\Users\heman\OneDrive\Black coffer\StopWords\*.txt") #searches all the txt files in stopwords folder

In [59]:
#appending all stopwords into one list

all_stop_words=[]

for i in stop_words_path:
            
            stop=open(f'{i}',"r")
            stop_list=stop.read().replace("\n",",").split(",")
            all_stop_words.extend(stop_list)
            stop.close()         
        
#Converting all stop words to small case       
all_stop_words=[x.lower() for x in all_stop_words]       

In [60]:
#Cleaning the stop words as they contain "|"

stripped_stop_words=[]
for i in all_stop_words:
    regex=re.search('(\w*\'*\w*)(\s*)(\|*\s*.*)',i)   #making a regex pattern so the words with | are stripped
    stripped_stop_words.append(regex.group(1)) 
       

In [61]:
#Personal pronouns

def personal_pronouns(split_body):
    pronouns_count=0
    for word in split_body:               #we count the no_of_pronouns in text which we took from before removing stopwords
        pronouns=['^(i?)(i?)$','^we$','^my$','^ours$','^us$']
        for pattern in pronouns:
            if re.search(pattern,word,re.M):
                pronouns_count+=1
    return pronouns_count


### Cleaning using stop words lists

In [62]:
#Cleaning the links with all the stop words
pro=[]

ID=link.URL_ID[0]

for i in range(37,148):
                                         #reading each text file
    body=open(f'C:\\Users\\heman\\OneDrive\\Black coffer\\{i}.txt',"r",encoding='utf-8')
    split_body=body.read().split()                             #converting it into list
    pro_count=personal_pronouns(split_body)
    pro.append(pro_count)
    body.close()
    clean_words=[word for word in split_body if word not in stripped_stop_words]  #cleaning the list by removing stop words
    clean_txt=" ".join(clean_words)                            #converting the list into string
    clean_link=open(f'{ID}.txt','w',encoding='utf-8')                           #rewriting the text file after removing stop words
    clean_link.write(clean_txt)
    clean_link.close()                                         #closing the text file
    ID+=1

### Dictionary of Positive and Negative words

In [63]:
# Master Dictionary of positive words

positive_words=set()
positive_open=open(r"C:\Users\heman\OneDrive\Black coffer\MasterDictionary\positive-words.txt",'r')
positive_list=positive_open.read().replace("\n",",").split(",")  #reading positive words text file and converting to list
for i in positive_list:
        positive_words.add(i)                                    #adding each word in list to dictionary
positive_words.remove("")        


In [64]:
# Master Dictionary of negative words

negative_words=set()
negative_open=open(r"C:\Users\heman\OneDrive\Black coffer\MasterDictionary\negative-words.txt",'r')
negative_list=negative_open.read().replace("\n",",").split(",")  #reading negative words text file and converting to list
for i in negative_list:
        negative_words.add(i)                                    #adding each word in list to dictionary
negative_words.remove("")  


### Extracting derived variables

In [65]:
#positive_score

def positive_score(word_tokens):                   #calculating positive score if word is found in positive dictionary
        pos_score=0
        for word in word_tokens:
            if word in positive_words:
                pos_score+=1                       #adding by +1 if word found in positive dictionary
        return pos_score
        
#negative_score

def negative_score(word_tokens):                  #calculating negative score if word is found in negative dictionary
        neg_score=0
        for word in word_tokens:
            if word in negative_words:
                neg_score-=1
        neg_score*=-1                            #multiplying by -1 to make the score a positive value
        return neg_score

In [66]:
# Polarity score

def polarity_score(word_tokens):      #calculating polarity score to check if text is positive or negative in nature
        pos_score=0
        for word in word_tokens:
            if word in positive_words:
                pos_score+=1
        neg_score=0
        for word in word_tokens:
            if word in negative_words:
                neg_score-=1
        neg_score*=-1          
        polarity=round((pos_score-neg_score)/((pos_score+neg_score)+0.000001),3)
        return polarity


In [67]:
#Subjectivity score

def subjectivity_score(word_tokens):#calculating subjectivity score by using positive and negative score
        pos_score=0
        for word in word_tokens:
            if word in positive_words:
                pos_score+=1
        neg_score=0
        for word in word_tokens:
            if word in negative_words:
                neg_score-=1
        neg_score*=-1      
        
        subj_score=round((pos_score+neg_score)/((len(word_tokens))+0.000001),3)
        return subj_score
    
   

### Analysis of readability

In [68]:
# Analysis of readability

##Average sentence length
def average_sentence_length(word_tokens,sent_tokens):     # checking the average sentence length
            avg_sent_len=round(len(word_tokens)/len(sent_tokens),3)
            return avg_sent_len


In [69]:
#syllable words count

def syllables_count(word):
    count = 0
    complex_count=0
    vowels = "aeiouy"                             #assigning all vowels and letter y
    for token in word:
        if token[0] in vowels:                    #reading each word in the list
            count += 1
        for index in range(1, len(token)):
            if token[index] in vowels and token[index - 1] not in vowels:   #checking if no two same vowels are consecutive
                count += 1
        if token.endswith("ed"or"es"):           #reducing the count by 1 if words end with 'ed' and 'es'
            count -= 1
        if count == 0:
            count += 1
    return count                                 #returning the count of syllables



In [70]:
#Complex words count

def complex_words_count(word):                  #counting no:of complex words 
    complex_word_count=0
    vowels = "aeiouy"
    for token in word:
        count=0
        if token[0] in vowels:
            count += 1
        for index in range(1, len(token)):
            if token[index] in vowels and token[index - 1] not in vowels:
                count += 1
        if token.endswith("ed"or"es"):
            count -= 1
        if count == 0:
            count += 1
           
        if count > 2: 
            complex_word_count+=1               #adding by 1 to count if no:of syllables in each word are more than 2
    return complex_word_count


In [71]:
#Percentage of complex words

def perc_complex_words(word_tokens):
            perc_compl=round(complex_words_count(word_tokens)/len(word_tokens),3)  #calc percentage of complex words
            return perc_compl
            

In [72]:
#Gunning Fog index

def fog_index(word_tokens,sent_tokens):          #calculating gunning fog index
    index=round(0.4*(average_sentence_length(word_tokens,sent_tokens)+perc_complex_words(word_tokens)),3)
    return index
   

### Average number of words per sentence 

In [73]:
# Avg number of words per sentence

def avg_words_per_sentence(word_tokens,sent_tokens):
            avg_words=round(len(word_tokens)/len(sent_tokens),3)   #calculating avg number of words in the sentence
            return avg_words
        

### Word Count 

In [74]:
#counting the no:of words left after remving stop words using nltk package

def cleaned_text_extended(word_tokens):
    from nltk.corpus import stopwords
    cleaned_word_extended=[word for word in word_tokens if word not in stopwords.words('english')]  #removing more stopwords
    punctuations='.!?,'
    cleaned_word_extended=[symbol for symbol in cleaned_word_extended if symbol not in punctuations] #removing punctuations
    return cleaned_word_extended

In [75]:
#counting no:of words in text after remving all stop words from nltk stopwords

def cleaned_nltk(word_tokens):
    count=len(cleaned_text_extended(word_tokens))
    return count


### Average word length

In [76]:
# Average word length

#we take the text from cleaned word extended
def avg_word_length(word_tokens):
    word_length=0
    for word in cleaned_text_extended(word_tokens):       #Iterating against each word and finding its length
        word_length+=len(word)

    avg_word_length=round(word_length/len(cleaned_text_extended(word_tokens)),3)  #calculating the average word length
    return avg_word_length


### Reading output file

In [77]:
output=pd.read_excel(r"C:\Users\heman\OneDrive\Black coffer\Output Data Structure.xlsx")
output.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Assigning values to dataframe

In [78]:
from nltk.tokenize import word_tokenize                 #importing word tokenizer
from nltk.tokenize import sent_tokenize                 #importing sentence tokenizer

row_index=0
                                                          #reading all text files in the folder 
for file in range(37,148):
    clean_text=open(f'C:\\Users\\heman\\OneDrive\\Black coffer\\{file}.txt',"r",encoding='utf-8')
    word_tokens=word_tokenize(clean_text.read())        #reading the txt file and storing word tokens as a list
    clean_text.close()
    clean_text=open(f'C:\\Users\\heman\\OneDrive\\Black coffer\\{file}.txt',"r",encoding='utf-8')
    sent_tokens=sent_tokenize(clean_text.read())        #reading the text file and storing sentence tokens as a list
    clean_text.close()
   
    positive_final=positive_score(word_tokens)          #checking the positive score
    
    
    negative_final=negative_score(word_tokens)          #checking the negative score
   
    
    polarity_final=polarity_score(word_tokens)  #checking the polarity score
   
    
    subject_final=subjectivity_score(word_tokens)    #Checking the subjectivity score
    
    
    avg_sent_len_final=average_sentence_length(word_tokens,sent_tokens)  #checking the avg length of sentence
   
    
    syllables_final=syllables_count(word_tokens)        #checking the no:of syllables
   
    
    complex_final=complex_words_count(word_tokens)      #checking the no:of complex words
   
    
    perc_complex_final=perc_complex_words(word_tokens)  #checking percentage of complex words
   
    
    fog_index_final=fog_index(word_tokens,sent_tokens)  #checking the gunning fox inex
   
    
    avg_words_sent_final=avg_words_per_sentence(word_tokens,sent_tokens)  #checking no:of avg words in a sentence
   
    
    clean_text_len_final=cleaned_nltk(word_tokens)#checking the length after cleaning using nltk stop words
   
    
    pronouns_final=personal_pronouns(word_tokens)
   
    
    avg_word_len_final=avg_word_length(word_tokens)     #checking the avg word length 
   
    
    #Assigning the claculated values to the dataframe
    
    output.loc[row_index] = pd.Series({'URL_ID':output.URL_ID[row_index],'URL':output.URL[row_index],
                                       'POSITIVE SCORE':positive_final, 'NEGATIVE SCORE':negative_final,
                                       'POLARITY SCORE':polarity_final,'SUBJECTIVITY SCORE':subject_final,
                                       'AVG SENTENCE LENGTH':avg_sent_len_final,'PERCENTAGE OF COMPLEX WORDS':perc_complex_final,
                                       'FOG INDEX':fog_index_final,'AVG NUMBER OF WORDS PER SENTENCE':avg_words_sent_final,
                                       'COMPLEX WORD COUNT':complex_final,'WORD COUNT':clean_text_len_final,
                                       'SYLLABLE PER WORD':syllables_final,'PERSONAL PRONOUNS':pro[row_index],
                                       'AVG WORD LENGTH':avg_word_len_final})
    row_index+=1
    

In [79]:
#Checking the data frame output
output=output.set_index('URL_ID')
output.head()

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
URL_ID,,,,,,,,,,,,,,
37,https://insights.blackcoffer.com/ai-in-healthc...,64.0,32.0,0.333,0.089,14.554,0.449,6.001,14.554,484.0,992.0,2489.0,1.0,7.498
38,https://insights.blackcoffer.com/what-if-the-c...,58.0,38.0,0.208,0.139,10.162,0.315,4.191,10.162,218.0,579.0,1332.0,6.0,6.922
39,https://insights.blackcoffer.com/what-jobs-wil...,64.0,37.0,0.267,0.108,11.024,0.450,4.590,11.024,422.0,833.0,2181.0,2.0,7.579
40,https://insights.blackcoffer.com/will-machine-...,60.0,27.0,0.379,0.119,8.738,0.343,3.632,8.738,252.0,637.0,1516.0,18.0,6.959
41,https://insights.blackcoffer.com/will-ai-repla...,56.0,23.0,0.418,0.087,12.667,0.355,5.209,12.667,324.0,797.0,1856.0,15.0,6.907


### Exporting data frame to excel



In [80]:
#giving file name

file_name='Output Data Structure.xlsx'     #giving a file name
output.to_excel(file_name)                 #output to excel
print("Exported Successfully")

Exported Successfully
